In [2]:
from fastai.text.all import *
path = untar_data(URLs.IMDB)

In [3]:
files = get_text_files(path, folders =['train', 'test', 'unsup'])

In [4]:
txt = files[0].open().read(); 
txt[:76]

'Once again Mr. Costner has dragged out a movie for far longer than necessary'

In [5]:
spacy = WordTokenizer()
toks = first(spacy([txt]))
print(coll_repr(toks, 30))

(#187) ['Once','again','Mr.','Costner','has','dragged','out','a','movie','for','far','longer','than','necessary','.','Aside','from','the','terrific','sea','rescue','sequences',',','of','which','there','are','very','few','I'...]


In [6]:
first(spacy(['The U.S. dollar lis1.00']))

(#4) ['The','U.S.','dollar','lis1.00']

In [7]:
tkn = Tokenizer(spacy)
print(coll_repr(tkn(txt), 31))

(#207) ['xxbos','xxmaj','once','again','xxmaj','mr','.','xxmaj','costner','has','dragged','out','a','movie','for','far','longer','than','necessary','.','xxmaj','aside','from','the','terrific','sea','rescue','sequences',',','of','which'...]


In [8]:
defaults.text_proc_rules

[<function fastai.text.core.fix_html(x)>,
 <function fastai.text.core.replace_rep(t)>,
 <function fastai.text.core.replace_wrep(t)>,
 <function fastai.text.core.spec_add_spaces(t)>,
 <function fastai.text.core.rm_useless_spaces(t)>,
 <function fastai.text.core.replace_all_caps(t)>,
 <function fastai.text.core.replace_maj(t)>,
 <function fastai.text.core.lowercase(t, add_bos=True, add_eos=False)>]

In [9]:
coll_repr(tkn('©   Fast.ai www.fast.ai/INDEX'), 31)

"(#11) ['xxbos','©','xxmaj','fast.ai','xxrep','3','w','.fast.ai','/','xxup','index']"

Subword Tokenization

In [10]:


txts = L(o.open().read() for o in files[:2000])

In [11]:
def subword(sz):
    sp = SubwordTokenizer(vocab_sz=sz)
    sp.setup(txts)
    return " ".join(first(sp([txt]))[:40])

In [12]:
subword(1000)

'▁O n ce ▁again ▁M r . ▁Co st n er ▁has ▁ d ra g g ed ▁out ▁a ▁movie ▁for ▁far ▁long er ▁than ▁ ne ce s s ar y . ▁A side ▁from ▁the ▁ ter'

In [13]:
subword(200)

'▁ O n ce ▁a g a in ▁ M r . ▁ C o s t n er ▁ h a s ▁ d ra g g ed ▁ o u t ▁a ▁movie ▁for ▁f ar ▁ l'

In [14]:
subword(10000)

'▁On ce ▁again ▁Mr . ▁Costner ▁has ▁dragged ▁out ▁a ▁movie ▁for ▁far ▁longer ▁than ▁necessary . ▁A side ▁from ▁the ▁terrific ▁sea ▁rescue ▁sequences , ▁of ▁which ▁there ▁are ▁very ▁few ▁I ▁just ▁did ▁not ▁care ▁about ▁any ▁of'

STEP-2: Numericalization

In [15]:


toks = tkn(txt)
print(coll_repr(tkn(txt), 31))

(#207) ['xxbos','xxmaj','once','again','xxmaj','mr','.','xxmaj','costner','has','dragged','out','a','movie','for','far','longer','than','necessary','.','xxmaj','aside','from','the','terrific','sea','rescue','sequences',',','of','which'...]


In [16]:
toks200 = txts[:200].map(tkn)
toks200[10]

(#146) ['xxbos','xxmaj','cage','plays','a','drunk','and','gets','high','critically','praise','.','xxmaj','elizabeth','xxmaj','shue','xxmaj','actually','has','to'...]

In [17]:
num = Numericalize()
num.setup(toks200)
coll_repr(num.vocab, 20)

"(#1968) ['xxunk','xxpad','xxbos','xxeos','xxfld','xxrep','xxwrep','xxup','xxmaj','the','.',',','a','and','of','to','is','it','i','in'...]"

In [18]:
nums = num(toks)[:20]; nums

TensorText([   2,    8,  349,  183,    8, 1177,   10,    8, 1178,   60, 1455,
              62,   12,   25,   28,  189,  957,   93,  958,   10])

In [19]:
' '.join(num.vocab[o] for o in nums)

'xxbos xxmaj once again xxmaj mr . xxmaj costner has dragged out a movie for far longer than necessary .'

In [31]:

#hide_input
stream = "In this chapter, we will go back over the example of classifying movie reviews we studied in chapter 1 and dig deeper under the surface. First we will look at the processing steps necessary to convert text into numbers and how to customize it. By doing this, we'll have another example of the PreProcessor used in the data block API.\nThen we will study how we build a language model and train it for a while."
tokens = tkn(stream)
bs,seq_len = 6,15
d_tokens = np.array([tokens[i*seq_len:(i+1)*seq_len] for i in range(bs)])
df = pd.DataFrame(d_tokens)


In [32]:
nums200 = toks200.map(num)

In [33]:
dl = LMDataLoader(nums200)

In [34]:
x,y = first(dl)
x.shape, y.shape

(torch.Size([64, 72]), torch.Size([64, 72]))

In [35]:
' '.join(num.vocab[o] for o in x[0][:20])

'xxbos xxmaj once again xxmaj mr . xxmaj costner has dragged out a movie for far longer than necessary .'

STEP -3: Training a Text Classifier

In [36]:
get_imdb = partial(get_text_files, folders=['train', 'test', 'unsup'])

In [42]:
dls_lm = DataBlock(
    blocks = TextBlock.from_folder(path, is_lm=True),
    get_items = get_imdb, splitter = RandomSplitter(0.1),
).dataloaders(path, path = path, bs=128, seq_len=80)

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\INDIA\\.fastai\\data\\imdb_tok\\counter.pkl'

In [28]:
dls_lm.show_batch(max_n=2)

NameError: name 'dls_lm' is not defined

STEP-4: Fine Tuning the Language Model

In [41]:
learn = language_model_learner(
    dls_lm, AWD_LSTM, drop_mult=0.3,
    metrics=[accuracy, Perplexity()]).to_fp16()

NameError: name 'dls_lm' is not defined

In [26]:
learn.fit_one_cycle(1, 2e-2)

NameError: name 'learn' is not defined

SAVING AND LOADING MODELS

In [27]:
learn.save('1epoch')

NameError: name 'learn' is not defined

In [40]:
learn = learn.load('1epoch')

NameError: name 'learn' is not defined

In [ ]:
learn.unfreeze()
learn.fit_one_cycle(10, 2e-3)

In [ ]:
learn.save_encode('finetuned')

Text Generation

In [39]:
text = "I am learning NLP and it is a bit hard but i like it."

n_words = 50

n_sentences = 3

preds = [learn.predict(text, n_words, temperature=0.75)
         for _ in range(n_sentences)]

NameError: name 'learn' is not defined

In [38]:
! pip install fastai


In [43]:
dls_clas = DataBlock(
    blocks=(TextBlock.from_folder(path, vocab=dls_lm.vocab),CategoryBlock),
    get_y = parent_label,
    get_items=partial(get_text_files, folders=['train', 'test']),
    splitter=GrandparentSplitter(valid_name='test')
).dataloaders(path, path=path, bs=128, seq_len=72)
     

NameError: name 'dls_lm' is not defined

In [44]:
nums_samp = toks200[:10].map(num)

In [45]:
nums_samp.map(len)

(#10) [207,314,267,378,304,156,197,187,162,229]